In [2]:
import os
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/anaconda3/bin/python3.7'
os.environ['PYSPARK_PYTHON'] = '/opt/anaconda3/bin/python3.7'
os.environ["SPARK_HOME"] = '/opt/cloudera/parcels/CDH/lib/spark'
import findspark
findspark.init()
from pyspark import SparkContext , SparkConf
from pyspark.sql import SparkSession,SQLContext,Row

from pyspark.streaming import StreamingContext

In [3]:
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Pysparkdf").getOrCreate()

1.Load moves.csv and cache the dataframe

In [4]:
movies_df=spark.read.csv("movies.csv",sep=',',inferSchema=True,header=True)

In [8]:
movies_df.persist()

DataFrame[movieId: int, title: string, genres: string]

In [17]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

2.Load ratings.csv and cache the dataframe

In [9]:
ratings_df=spark.read.csv("ratings.csv",sep=',',inferSchema=True,header=True)

In [10]:
ratings_df.persist()

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [11]:
ratings_df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



3.Find the number of records in movies dataframe

In [15]:
movies_df.count()

9742

4.Find the number of records in ratings dataframe

In [16]:
ratings_df.count()

100836

In [23]:
ratings_df.select("userId").distinct().show()

+------+
|userId|
+------+
|   148|
|   463|
|   471|
|   496|
|   243|
|   392|
|   540|
|    31|
|   516|
|    85|
|   137|
|   251|
|   451|
|   580|
|    65|
|   458|
|    53|
|   255|
|   481|
|   588|
+------+
only showing top 20 rows



In [26]:
movies_df.createOrReplaceTempView("Movies")
ratings_df.createOrReplaceTempView("Ratings")

In [27]:
Comb = spark.sql("SELECT * from Ratings inner join Movies on Ratings.userId=Movies.movieId ")

In [28]:
Comb.show()

+------+-------+------+---------+-------+----------------+--------------------+
|userId|movieId|rating|timestamp|movieId|           title|              genres|
+------+-------+------+---------+-------+----------------+--------------------+
|     1|      1|   4.0|964982703|      1|Toy Story (1995)|Adventure|Animati...|
|     1|      3|   4.0|964981247|      1|Toy Story (1995)|Adventure|Animati...|
|     1|      6|   4.0|964982224|      1|Toy Story (1995)|Adventure|Animati...|
|     1|     47|   5.0|964983815|      1|Toy Story (1995)|Adventure|Animati...|
|     1|     50|   5.0|964982931|      1|Toy Story (1995)|Adventure|Animati...|
|     1|     70|   3.0|964982400|      1|Toy Story (1995)|Adventure|Animati...|
|     1|    101|   5.0|964980868|      1|Toy Story (1995)|Adventure|Animati...|
|     1|    110|   4.0|964982176|      1|Toy Story (1995)|Adventure|Animati...|
|     1|    151|   5.0|964984041|      1|Toy Story (1995)|Adventure|Animati...|
|     1|    157|   5.0|964984100|      1

In [43]:
spark.sql("SELECT avg(rating),count(*) from Rating group by movieId  ").show()

+-----------+--------+
|avg(rating)|count(1)|
+-----------+--------+
|        2.5|       1|
|        4.5|       1|
|        3.0|       1|
|        4.0|       1|
|        3.0|       1|
|        3.5|       1|
|        4.0|       2|
|        3.0|       1|
|        5.0|       1|
|        2.0|       1|
+-----------+--------+



In [47]:
spark.sql("SELECT max(rating) from Ratings group by movieId ").show()

+-----------+
|max(rating)|
+-----------+
|        5.0|
|        5.0|
|        5.0|
|        5.0|
|        5.0|
|        4.5|
|        4.5|
|        5.0|
|        4.0|
|        5.0|
|        5.0|
|        4.0|
|        3.5|
|        5.0|
|        3.0|
|        4.0|
|        5.0|
|        4.5|
|        5.0|
|        4.5|
+-----------+
only showing top 20 rows



In [ ]:
spark.sql("SELECT movieId,avg(rating),count(rating) from Ratings")